In [100]:
### IMPORTS

import numpy
import scipy,scipy.optimize,scipy.spatial,scipy.spatial.distance,scipy.stats
import json
import sqlite3
from collections import *
from enum import Enum
numpy.random.seed(1)

In [101]:
### CONSTANTS

# Names of goods
Goods = ["Credits","Grain","Food","Fuel","Diamonds","null"]


In [102]:
class BoundedAdaptiveNormalDistribution:
    NumSamples = 100
    
    def __init__(self, mean, variance, lower=None, upper=None):
        self.mean = mean
        self.variance = variance
        self.lower = lower
        self.upper = upper
        
    def __repr__(self):
        return str(self.mean)
        
    def draw(self):
        while True:
            value = scipy.stats.norm.rvs(loc=self.mean, scale=self.variance)
            if self.lower and self.lower > value:
                continue
            if self.upper and self.upper < value:
                continue
            return value
        
    def update(self,newValue):
        NumSamples = BoundedAdaptiveNormalDistribution.NumSamples
        oldMean = self.mean;
        self.mean = ((self.mean * (NumSamples - 1)) + newValue) / NumSamples
        self.variance = (self.variance*(NumSamples-1) + ((newValue - oldMean)*(newValue - self.mean))) / NumSamples;

class Agent:
    def __init__(self, loc):
        self.loc = numpy.copy(loc)
        
    def __repr__(self):
        return self.__dict__.__repr__()

class Trader(Agent):
    def __init__(self, loc):
        Agent.__init__(self, loc)
        self.inventory = numpy.copy([1000,0,0,0,0,0])

class Factory(Trader):
    def __init__(self,loc,productionCost,output,capacity):
        Trader.__init__(self,loc)
        self.productionCost = numpy.copy(productionCost)
        self.output = numpy.copy(output)
        self.capacity = capacity
        self.demand = 0

    def getEpochsOfProduction(self):
        epochs = 100
        for i in range(0,len(self.productionCost)):
            if i == 0:
                continue # You can go negative in money with no penalty for now
            if self.productionCost[i] == 0:
                continue
            epochs = min(epochs, self.inventory[i] // self.productionCost[i] // self.capacity)
        return epochs
    
    def getCurrentProduction(self):
        currentProduction = self.capacity
        for i in range(0,len(self.productionCost)):
            if i == 0:
                continue # You can go negative in money with no penalty for now
            if self.productionCost[i] == 0:
                continue
            currentProduction = min(currentProduction, self.inventory[i] // self.productionCost[i])
        return currentProduction
    
    def produce(self):
        production = min(2,self.getCurrentProduction())
        self.inventory -= self.productionCost * production
        self.inventory += self.output * production
        

class Merchant(Trader):
    def __init__(self,loc):
        Trader.__init__(self,loc)

class Contract():
    def __init__(self,source,destination,good,quantity,pricePerUnit):
        self.source = source
        self.destination = destination
        self.good = good
        self.quantity = quantity
        self.pricePerUnit = pricePerUnit

In [103]:
## HELPER FUNCTIONS

def spawnLocation():
    return numpy.array([numpy.random.randint(0,100), numpy.random.randint(0,100)])


In [104]:
PlanetLocation = spawnLocation()
Factories = [
    Factory(spawnLocation(), [1,0,0,0,0,0], [0,1,0,0,0,0], 100),
    Factory(spawnLocation(), [1,1,0,0,0,0], [0,0,1,0,0,0], 100),
    Factory(spawnLocation(), [5,0,0,0,0,0], [0,0,0,1,0,0], 100),
    Factory(spawnLocation(), [10,0,0,0,0,0], [0,0,0,0,1,0], 100),
    Factory(PlanetLocation, [0,0,1,0,0,0], [10,0,0,0,0,0], 100),
    Factory(PlanetLocation, [0,0,0,1,0,0], [40,0,0,0,0,0], 50),
    Factory(PlanetLocation, [0,0,0,0,1,0], [160,0,0,0,0,0], 25),
]
Contracts = []
EstimatedPrices = [BoundedAdaptiveNormalDistribution(1,0)]
for x in range(1,len(Goods)):
    EstimatedPrices.append(BoundedAdaptiveNormalDistribution(0,10,1,1000))

print(Factories)

for epoch in range(0,1):
    for turn in range(0,10000):
        Contracts = []
        for i in range(0,1):
            # First, draw to estimate the prices of items
            prices = [1,]
            for i2 in range(1,len(Goods)):
                prices.append(EstimatedPrices[i2].draw())
            for f in Factories:
                currentProduction = f.getCurrentProduction()
                remainingProduction = f.capacity - currentProduction
                if remainingProduction > 0:
                    # Next, decide if more production makes sense given these prices
                    costValue = numpy.dot(f.productionCost,prices)
                    outputValue = numpy.dot(f.output,prices)
                    if costValue < outputValue:
                        # Try to find suppliers for missing goods
                        for goodToProduce in range(1,len(Goods)):
                            need = (f.productionCost[goodToProduce] * f.capacity) - f.inventory[goodToProduce]
                            if need > 0:
                                for supplier in Factories:
                                    if supplier.inventory[goodToProduce] > 0 and \
                                    supplier.productionCost[goodToProduce] == 0: # For now, don't buy from someone who needs the good in their production
                                        # Buy goods from supplier
                                        quantityToBuy = 1 # Don't try to max out in case your price estimate is bad
                                        #quantityToBuy = min(need, supplier.inventory[goodToProduce])
                                        f.inventory[goodToProduce] += quantityToBuy
                                        supplier.inventory[goodToProduce] -= quantityToBuy
                                        f.inventory[0] -= quantityToBuy * prices[goodToProduce]
                                        supplier.inventory[0] += quantityToBuy * prices[goodToProduce]
                                        Contracts.append(Contract(supplier,f,goodToProduce,quantityToBuy,prices[goodToProduce]))

        # Produce when it makes sense
        
        # First, draw to estimate the prices of items
        prices = [1,]
        for i2 in range(1,len(Goods)):
            prices.append(EstimatedPrices[i2].draw())
        for f in Factories:
            # Next, decide if more production makes sense given these prices
            costValue = numpy.dot(f.productionCost,prices)
            outputValue = numpy.dot(f.output,prices)
            if costValue < outputValue:
                # Produce
                f.produce()

        # Based on the contracts, adjust the prices
        for c in Contracts:
            EstimatedPrices[c.good].update(c.pricePerUnit)
            
        if turn % 100 == 0:
            for c in Contracts:
                print("Updating from contract: ",Goods[c.good],c.pricePerUnit)
            print("New prices: ",EstimatedPrices)
            print("Factories")
            for f in Factories:
                print(f.inventory)
            print('done')
    

[{'loc': array([72,  9]), 'productionCost': array([1, 0, 0, 0, 0, 0]), 'output': array([0, 1, 0, 0, 0, 0]), 'capacity': 100, 'inventory': array([1000,    0,    0,    0,    0,    0]), 'demand': 0}, {'loc': array([75,  5]), 'productionCost': array([1, 1, 0, 0, 0, 0]), 'output': array([0, 0, 1, 0, 0, 0]), 'capacity': 100, 'inventory': array([1000,    0,    0,    0,    0,    0]), 'demand': 0}, {'loc': array([79, 64]), 'productionCost': array([5, 0, 0, 0, 0, 0]), 'output': array([0, 0, 0, 1, 0, 0]), 'capacity': 100, 'inventory': array([1000,    0,    0,    0,    0,    0]), 'demand': 0}, {'loc': array([16,  1]), 'productionCost': array([10,  0,  0,  0,  0,  0]), 'output': array([0, 0, 0, 0, 1, 0]), 'capacity': 100, 'inventory': array([1000,    0,    0,    0,    0,    0]), 'demand': 0}, {'loc': array([37, 12]), 'productionCost': array([0, 0, 1, 0, 0, 0]), 'output': array([10,  0,  0,  0,  0,  0]), 'capacity': 100, 'inventory': array([1000,    0,    0,    0,    0,    0]), 'demand': 0}, {'loc':